In [1]:
#For experimenting, evaluate this cell and then run the cell beneath it

from collections import defaultdict, Counter
from itertools import *

graph_cache = {}
simple_delta_names = False
delta_count = 0
found_interesting_case = False
def create_game_graph(num_of_players, num_of_strategies):
    #The gamegraph is isomorphic to hamming graph H(p,q) where 
    #p == num_of_players
    #q == num_of_strategies
    key = (num_of_players, num_of_strategies)
    if key in graph_cache:
        return graph_cache[key]
    G = graphs.HammingGraph(num_of_players, num_of_strategies)
    all_vars = []
    vertex_to_index = {}
    #Set variables for each vertex
    #ex, in the 3x3 game graph case, the first vertex has the variables
    #v_0_0, v_0_1, v_0_2
    for i, vertex in enumerate(G.vertices()):
        vertex_to_index[vertex]=i
        vars = [var('v_{0}_{1}'.format(i,j)) for j,_ in enumerate(vertex)]
        for v in vars:
    #        assume(v,'integer')
            all_vars.append(v)
            G.set_vertex(vertex, vars)
    vertex_to_deltas=defaultdict(list)
    graph_cache[key] = (G, all_vars)
    return (G, all_vars)

def create_delta(G, u,v):
    global delta_count, simple_delta_names
    coord  = 0
    for j, (x,y) in enumerate(zip(u, v)):
        if x!=y:
            #this is the coordinate we are adjacent on
            coord = j
    if simple_delta_names:
        delta = var("d_{0}".format(delta_count))
        delta_count+=1
    else:
        delta = var("d_{0}_v_{1}_{2}".format(coord, '_'.join(map(str,u)),'_'.join(map(str,v))))
    #Any delta we actually use in an equation will be non-negative
    assume(delta>0)
    G.set_edge_label(u,v,(coord, delta))
    return coord, delta
     #   all_vars.append(delta)
              #      all_vars.append(delta)
    G.set_edge_label(u,v, (coord, delta))
    
def create_deltas_for_vertex(G, v):
    for u,v in G.edges_incident([v]):
        coord, delta = create_delta(u,v)
        G.set_edge_label(u,v,coord, delta)
    
def create_zero_sum_equations(G, include_matrix_rows = False):
    zero_sums = []
    matrix_rows = []
    #Each vertex is zero sum
    vertices = list(G.vertices())
    length = len(vertices)*len(vertices[0]) +1
    for i, vertex in enumerate(G.vertices()):
        vars = G.get_vertex(vertex)
        qe = reduce(lambda x,y:x+y, vars)
        zero_sums.append(qe==0)   
        
        row = [0 for _ in range(length)]
        #Find the index matching with this vertex and place a 1 for each direction
        index = i*len(vertex)
        for displace in range(len(vertex)):
            row[index+displace] = 1
        matrix_rows.append(row)
        
    if include_matrix_rows:
        return (zero_sums, matrix_rows)
    return zero_sums

#This version of create_zero_sum_equations will only produce zero-sum rows for
#variables involved in this configuration
#This can potentially save time for large graphs with relatively small number of 
def create_zero_sum_equations_specific(G, configuration, include_matrix_rows = False):
    zero_sums = []
    matrix_rows = []
    #Each vertex is zero sum
    
    vertices = list(G.vertices())
    vertex_to_index = {}
    for i,vertex in enumerate(vertices):
        vertex_to_index[vertex]=i
    seen = set()
    length = len(vertices)*len(vertices[0]) +1
    for nash_equilibrium in configuration:
        for u,v,_ in G.edges_incident(nash_equilibrium):
            for vertex in (u,v):
                if vertex in seen:continue
                seen.add(vertex)
                vars = G.get_vertex(vertex)
                qe = reduce(lambda x,y:x+y, vars)
                zero_sums.append(qe==0)
                row=[0 for _ in range(length)]
                i = vertex_to_index[vertex]
                index = i*len(vertex)
                for displace in range(len(vertex)):
                    row[index+displace] = 1
                matrix_rows.append(row)
    if include_matrix_rows:
        return (zero_sums, matrix_rows)
    return zero_sums

def axis_key(vertex, index):
    return tuple([index] + [c for j,c in enumerate(vertex) if j!=index])

def create_delta_equations(G, configuration, include_matrix_rows = False):
    deltas = []
    vertex_to_deltas = {}
    
    vertices = list(G.vertices())
    length = len(vertices)*len(vertices[0]) +1
    matrix_rows = []
    
    vertex_to_index = {}
    canonicals = {}
    for nash in configuration:
        for index, v in enumerate(nash):
            key = axis_key(nash, index)
            if key not in canonicals:
                canonicals[key] = nash
    print("Canonicals ", canonicals,end="\r\n")
            
    for i, v in enumerate(vertices):
        vertex_to_index[v] = i
    tuple_size = len(vertices[0])
    
    #Each edge's delta is equal to the difference of its matching vertex elements
    for nash_equilibrium in configuration:
        for u,v, label in G.edges_incident([nash_equilibrium]):
            if type(label) is tuple:
                coord, delta = label
            else:
                coord, delta = create_delta(G,u,v)
            u_is_nash = u in configuration
            v_is_nash = v in configuration
            if not u_is_nash and not v_is_nash:
                #Two non-nashes can have any difference, doesn't matter
                continue
            
            #get the variables that are specifically adjacent for the two
            #vertices
            u_var = G.get_vertex(u)[coord]
            v_var = G.get_vertex(v)[coord]
            u_var_coord = vertex_to_index[u]*tuple_size+coord
            v_var_coord = vertex_to_index[v]*tuple_size+coord
            row = [0 for _ in range(length)]
            #If both are nash, we set the equation to 0
            #If only one is nash, we set the equation to = the edge delta
            if u_is_nash:

                if not v_is_nash:
                    if u != canonicals[axis_key(u, coord)]: continue
                    difference = u_var - v_var == delta
                    row[u_var_coord] = 1
                    row[v_var_coord] = -1
                    row[-1] = delta
                else:
                    if canonicals[axis_key(u, coord)] not in (u,v): continue
                    difference = u_var - v_var == 0
                    row[u_var_coord] = 1
                    row[v_var_coord] = -1
                
            else:
                    if v != canonicals[axis_key(v, coord)]: continue
                    difference = v_var - u_var == delta
                    row[u_var_coord] = -1
                    row[v_var_coord] = 1
                    row[-1] = delta

            matrix_rows.append(row)
            deltas.append(difference)
        
    if include_matrix_rows:
        return (deltas, matrix_rows)
    return deltas


def is_feasible_rigorous(G, all_vars, configuration):
    zero_sums = create_zero_sum_equations_specific(G, configuration)
    deltas = create_delta_equations(G, configuration)
    extra = [deq.rhs() > 0 for deq in deltas if deq.rhs()!=0]
    ds = [deq.rhs() for deq in deltas if deq.rhs() !=0 ]
    vv = all_vars + ds
    solutions = solve(zero_sums + deltas + extra, *vv)
    return solutions is not None and len(solutions) > 0

def approx_solution_rigorous(G, all_vars, configuration):
    zero_sums = create_zero_sum_equations_specific(G, configuration)
    deltas = create_delta_equations(G, configuration)
    extra = [deq.rhs() > 0 for deq in deltas if deq.rhs()!=0]
    ds = [deq.rhs() for deq in deltas if deq.rhs() !=0 ]
    vv = all_vars + ds
    print(extra)
    solutions = solve(zero_sums + deltas + extra, *vv, explicit_solutions=True)
    if solutions is None or len(solutions)==0:
        return []
    solution = solutions[0]
    output = "FindInstance[{"+ ",".join(str(eq) for eq in solution)+"},"
    output += "{" + ",".join(str(v) for v in vv) + "}"
    output += "]"
    output = output.replace('_','u')
    return output
  #  if solutions is not None and len(solutions)>0:
   #     return [[s[v].n(30) for v in vv] for s in solutions]
    return []

    
def is_feasible_scratch_rigorous(num_of_players, num_of_strategies, configuration):
    G, all_vars = create_game_graph(num_of_players, num_of_strategies)
    return is_feasible_rigorous(G, all_vars, configuration)

def approx_solution_rigorous_scratch(num_of_players, num_of_strategies, configuration):
    G, all_vars = create_game_graph(num_of_players, num_of_strategies)
    return approx_solution_rigorous(G, all_vars, configuration)

def test_feasible1():
    G, vars = create_game_graph(3,3)
    #One vertex as nash and all other non-nash is feasible
    configuration = [list(G.vertices())[0]]
    print(is_feasible_rigorous(G, vars, configuration))

def test_feasible2():
    G, vars = create_game_graph(3,3)
    #All but one vertex being nash is infeasible
    configuration = list(G.vertices())[0:25]
    print(is_feasible_rigorous(G, vars, configuration))
    
def show_with_config(G, configuration):
    vertex_colors = defaultdict(list)
    for v in G.vertices():
        if v in configuration:
            vertex_colors["#00FF00"].append(v)
    G.show(vertex_colors = vertex_colors)
    
def display_3x3_game_graph(G, configuration):
    ##Display a 3 player 3 strategy game 
    ##It is broken into three layers to make it easier to visualize
    
    
    #to make things neater, let's only draw the edges for 
    #vertices with distance 1
    simple_edges = []
    for edge in G.edges():
        u, v, (coord, label) =edge
        dist = sum(abs(u_c - v_c) for u_c, v_c in zip(u,v))
        
        if dist==1:
            simple_edges.append( edge)
            
    top_layer = G.subgraph(vertices = list(G.vertices())[0:9], edges = simple_edges)
    show_with_config(top_layer, configuration)
    mid_layer = G.subgraph(vertices = list(G.vertices())[9:18], edges=simple_edges)
    show_with_config(mid_layer, configuration)
    bot_layer = G.subgraph(vertices = list(G.vertices())[18:], edges = simple_edges)
    show_with_config(bot_layer, configuration)
    
def test_display_game_graph(config=None):
    G, vars = create_game_graph(3,3)
#    configuration = [list(G.vertices())[0]]
    configuration = {(3, 0, 2), (2, 0, 0), (0, 0, 0), (1, 2, 0), (3, 1, 2)}# ((0, 1, 1), (1, 0, 1))
    display_3x3_game_graph(G, configuration)
    
#test_display_game_graph()
#test_feasible1()
#approx_solution_rigorous_scratch(3,3,{(0, 1, 1), (1, 2, 1), (0, 2, 0), (1, 2, 0), (0, 1, 0), (0, 1, 2), (1, 0, 0), (1, 0, 1), (1, 1, 2), (2, 1, 2), (0, 2, 1), (2, 0, 0)})
#is_feasible_quick(3,3,{(0, 1, 1), (1, 2, 1), (0, 2, 0), (1, 2, 0), (0, 1, 0), (0, 1, 2), (1, 0, 0), (1, 0, 1), (1, 1, 2), (2, 1, 2), (0, 2, 1), (2, 0, 0)})

import random
def search_for_random_subgraph(num_of_players, num_of_strategies, p=0.3, num_of_iterations=10, max_attempts=None):
    G, all_vars = create_game_graph(num_of_players, num_of_strategies)
    attempt = 0
    while True:
        configuration = set()
        
        if max_attempts and attempt == max_attempts-1:
            break
        print("Attempt", attempt, end='\r\n')
        attempt+=1
        i = 0
        while i < num_of_iterations:
            H = G.random_subgraph(p)
            if H.num_verts() <= 1:
                continue
            i+=1
            verts = list(H.vertices())
            configuration.add(random.choice(verts))
            if random.randint(0,100)>33: 
                configuration.add(random.choice(verts))
        print(configuration)
        if is_feasible_quick(G, all_vars, configuration):
            return configuration
        print("Checked feasible",end='\r\n')
    
def search_awhile(num_of_players, num_of_strategies, solutions=100, p=0.2, num_of_iterations=10):
    for i in range(solutions):
        config = None
        while not config:
            config = search_for_random_subgraph(num_of_players, num_of_strategies, p=p, num_of_iterations=num_of_iterations, max_attempts=20)
        print("Found:", config,"Size:",len(config))
        find_configs_that_need_algorithm(num_of_players,num_of_strategies,[config])
#search_awhile(4,4,num_of_iterations=8)

#Algorithm Helpers


def convert_eqs_to_solvable_form(G, qes):
    deltas = []
#    for u,v,(coord, delta) in G.edges():
#        deltas.append(delta)
    deltas = [qe[-1] for qe in qes if qe[-1]!=0]
        
    #Create a multipolynomial ring for this delta
    R_ = PolynomialRing(QQ,','.join(map(str, deltas)))
    deltas_ = R_.gens()
    convert_map = {}
    for d, d_ in zip(deltas, deltas_):
        convert_map[str(d)] = d_
        

    system = qes  
  #  system = [create_matrix_row(qe, v_index) for qe in qes]
  #  print(system)
    return matrix([[x if str(x) not in convert_map else convert_map[str(x)] for x in qe] for qe in system])

def convert_multivariate_eqs_to_vars(qes):
    delta_map= {}
    eqs = []
    for qe in qes:
        deltas = get_deltas(qe)
        row = []
        for pos_d in deltas['+']:
            converted_d = None
            if pos_d in delta_map:
                converted_d = delta_map[pos_d]
            else:
                converted_d = var(str(pos_d))
            assume(converted_d>0)
            delta_map[pos_d] = converted_d
            row.append(converted_d)
        for neg_d in deltas['-']:
            converted_d = None
            if neg_d in delta_map:
                converted_d = delta_map[neg_d]
            else:
                converted_d = var(str(neg_d))
                delta_map[neg_d] = converted_d
            assume(converted_d>0)
            row.append(-1*converted_d)
        if len(row)==1:
            eqs.append(row[0])
        else:
            eqs.append(reduce(lambda x,y:x+y,row))
    return eqs, delta_map.values()
            
    
def test_conversion():
    G, all_vars = create_game_graph(3,3)
    zero_sums, zero_sum_matrix = create_zero_sum_equations(G, True)
    configuration = [list(G.vertices())[0]]
    deltas, deltas_matrix = create_delta_equations(G, configuration, True)
    eqs = zero_sum_matrix + deltas_matrix
    eqs_ = convert_eqs_to_solvable_form(G, eqs)
    print(matrix(eqs_))

def convert_to_delta_map(qe):
    m = {'+':{}, '-':{}}
    for v in qe.variables():
        coefficient = qe.coefficient(v)
        if coefficient > 0:
            m['+'][str(v)] = coefficient
        else:
            m['-'][str(v)] = -1*coefficient
    return m

def delta_map_str(qe):
    pos = '+'.join(qe['+'].keys())
    neg = '-'.join(qe['-'].keys())
    return pos + '-' + neg

def get_var(eq, var):
    neg = eq['+']
    pos = eq['-']
    return -neg[var] if var in neg else pos[var] if var in pos else 0


def ensure_var(eq, var):
    if var in eq['+']: return
    if var in eq['-']: return
    eq['+'][var]=0

#Modify target_eq with source_eq per the Dines alogirhtm
def merge(source_eq, target_eq, all_vars):
    new_eq = {}
#    for key, _ in source_eq['+'].items():
#        ensure_var(target_eq, key)
#    for key, _ in source_eq['-'].items():
#        ensure_var(target_eq, key)
    #Book keeping to make sure that source_eq knows about variables in target_eq
#    for key, _ in target_eq['+'].items():
#        ensure_var(source_eq, key)
#    for key, _ in target_eq['-'].items():
#        ensure_var(source_eq, key)
    for key in all_vars:
        ensure_var(source_eq, key)
    
    for vari, coefficienti in source_eq['+'].items():
        for varj, coefficientj in source_eq['-'].items():
            coefficientj = -coefficientj
            target_coefficienti = get_var(target_eq, vari)
            target_coefficientj = get_var(target_eq, varj)
            #Find the new coefficient per Dines algorithm
            new_coefficient = ((coefficienti * target_coefficientj) - (coefficientj*target_coefficienti))
            #Create a new variable to track this
            key = vari + varj if vari<varj else varj+vari
            if key in new_eq:
                new_eq[key]+= new_coefficient
            else:
                new_eq[key] = new_coefficient

    new_eq_final = {'+':{}, '-':{}}
    for var, coefficient in new_eq.items():
        if coefficient == 0: continue
        sign = '+' if coefficient >=0 else '-'
        new_eq_final[sign][var] = coefficient if sign=='+' else -1*coefficient

    return new_eq_final

def get_all_vars(eqs):
    all_vars = set()
    for e in eqs:
        for sign, vs in e.items():
            for v in vs:
                all_vars.add(v)

    return all_vars

opposite = {'+':'-', '-':'+'}

def sign_matched_eqs(eq1,eq2):
    syncmap = {}
    for sign, vs in eq1.items():
        opp = eq2[opposite[sign]]
        same = eq2[sign]
        synced = None
        for v in vs:
            if v in opp:
                if synced is None:
                    synced = False
                elif synced:
                    #We're apparently not in sync
                    return False
            elif v in same:
                if synced is None:
                    synced = True
                elif not synced:
                    #We were not suppose to be in sync :( 
                    return False
        if synced is not None: syncmap[sign] = synced
    
    return len(syncmap)<2 or syncmap['+']==syncmap['-']
                
                    
                    
            
        
        
        


#Does this equation have a solution?
def ok(eq):
    return len(eq['+'])>0 and len(eq['-'])>0

def ok_pairwise(es):
    for i,eq1 in enumerate(es):
        for j,eq2 in enumerate(es):
            if i==j: continue
            if eq1.keys() != eq2.keys():
                return False
    return True
def big_coefficient(e):
    for sign, vars in e.items():
        for v, coeff in vars.items():
            if coeff>1:
                return True
    return False


#Does this system of equations have a solution?
#NOTE this is currently inefficient
def has_positive_solution(es, max_iter = 0):
    iter = 0
   # for eq1, eq2 in combinations(es, 2):
    #    if not sign_matched_eqs(eq1,eq2):
     #       print("Not sign matched:", eq1, eq2, end="\r\n")
      #      break
    while len(es)>1:
        iter+=1
        if max_iter > 0 and iter > max_iter:
            return True
        print(iter, end="\r\n")
        for e in es:
            print(delta_map_str(e), end="\r\n")
        all_vars = get_all_vars(es)
        if any(big_coefficient(e) for e in es):
            print("Big coefficient!", end="\r\n")
        #If there are equations that don't have positive and negative vars
        #then we're done
        if any(not ok(e) for e in es):
            return False
        if not ok_pairwise(es):
            return False
        #Pop off the top and merge it with the rest of the system
        rep = es.pop()
        for i, e in enumerate(es):
            es[i] = merge(rep, e, all_vars)
    return all(ok(e) for e in es)

    
def get_deltas(qe):
    #Returns from equation the deltas in a map
    #all positive deltas are mapped to '+'
    #all negative deltas are mapped to '-'
    vars = qe.variables()
    m = defaultdict(list)
    for v in vars:
        coefficient = qe.coefficient(v)
        if coefficient > 0:
            m['+'].append(v)
        else:
            m['-'].append(v)
    return m
def get_deltas_unsigned(qe):
    #Returns from equation the deltas
    vars = qe.variables()
    return vars

def has_delta(x, delta):
    #Returns '+' if delta is present in equation x's positive deltas
    #Returns '-' if delta is present in equation x's negative deltas
    #Returns False is it is not present.
    deltas = get_deltas(x)
    if delta in deltas['+']: return '+'
    if delta in deltas['-']: return '-'
    return False

def make_pos(x, delta):
    #Returns a copy of x such that delta is positive
    deltas = get_deltas(x)
    if delta in deltas['-']: return -x
    return x

def algorithm(eqs):
    #In Progress
    eqs_ = list(eqs)
    partitions = []
    while len(eqs_)>0:
        #Select an equation
        rep = eqs_[0]
        rep_deltas = get_deltas(rep)
        #Select a positive delta d_z
        d_z = rep_deltas['+'][0]\
        #Find those with that d_z
        partition = [x for x in eqs_ if has_delta(x, d_z)]
        #Make sure d_z is positive for everyone
        partition_ = [make_pos(x,d_z) for x in partition]
        print(partition_)
        partitions.append((d_z, partition_))
        eqs_ = [x for x in eqs_ if x not in partition]
    eliminated = set()

    for p, (d_z, partition) in enumerate(partitions):
        deleted = []
        i = 0
        print("Doing partition",p, partition)
        #Scan down this partition
        while i < len(partition):
            eq = partition[i]
            print("Checking eq",i,eq)
            #select a negative d
            rep_deltas = get_deltas(eq)
            d_pumps = [d for d in get_deltas(eq)['-'] if d not in deleted]
            if len(d_pumps)==0:
                print("RESTART")
                ###
            #TODO replace with selecting a different delta in the event this was a bad choice
            d_pump = d_pumps[0]
            #is this in someone else's negatives? We should return and try a different delta
            for j, eq2 in enumerate(partition[0:i]):
                if has_delta(eq, d_pump)=='-':
                    print("Go back up")
                    partition[i],partition[j] = eq2, eq
                    i = j
                    deleted = deleted[0:i]
                    break
            else:
                i+=1
    print("DONE")
    
def in_rref(M):
    cols = M.ncols()
    for r, row in enumerate(M):
        leading_pos = M.nonzero_positions_in_row(r)
        if len(leading_pos) == 0:
            continue
        if len(leading_pos)==1 and row[-1] != 0:
            continue
        leading_pos = leading_pos[0]
        others = M.nonzero_positions_in_column(leading_pos)
        if len(others) > 1:
            return False
    return True

def get_matrix(num_of_players, num_of_strategies, configuration):
    G, all_vars = create_game_graph(num_of_players,num_of_strategies)
    zero_sums, zero_sum_matrix = create_zero_sum_equations(G, True)
    deltas, deltas_matrix = create_delta_equations(G, configuration, True)
    eqs = zero_sum_matrix + deltas_matrix
    eqs_ = convert_eqs_to_solvable_form(G, eqs)
    return matrix(eqs_)

def test_row_reduction(configuration = None):

    G, all_vars = create_game_graph(3,3)
    
    zero_sums, zero_sum_matrix = create_zero_sum_equations(G, True)
    if configuration:
        deltas, deltas_matrix = create_delta_equations(G, configuration, True)
        eqs = zero_sum_matrix + deltas_matrix
        eqs_ = convert_eqs_to_solvable_form(G, eqs)
        ds = set([row[-1] for row in eqs_ if row[-1]!=0])
        eqs_rref = matrix(eqs_).echelon_form()
        ds_rref = set([row[-1] for row in eqs_rref if row[-1]!=0])
        for d in ds:
            print(d)
        print("=====")
        for d in ds_rref:
            print(d)
        print("=======")
        print([row[-1] for row in matrix(eqs_).echelon_form()])
        print("=====")
        
        return
         
def test_algorithm(configuration):
    G, all_vars = create_game_graph(3,3)
    
    zero_sums, zero_sum_matrix = create_zero_sum_equations(G, True)
    deltas, deltas_matrix = create_delta_equations(G, configuration, True)
    eqs = zero_sum_matrix + deltas_matrix
    eqs_ = convert_eqs_to_solvable_form(G, eqs)
    ds = set([row[-1] for row in eqs_ if row[-1]!=0])
    eqs_rref = matrix(eqs_).echelon_form()
    ds_rref = set([row[-1] for row in eqs_rref if row[-1]!=0 and len(set(row))==2])
    deltas_for_algorithm = list(ds_rref)
    algorithm(deltas_for_algorithm)
 
def gauss_method(M,rescale_leading_entry=False, augmented_column=True, deltas=None, printOutput=True):
    """Describe the reduction to echelon form of the given matrix of rationals.

    M  matrix of rationals   e.g., M = matrix(QQ, [[..], [..], ..])
    rescale_leading_entry=False  boolean  make the leading entries to 1's

    Returns: None.  Side effect: M is reduced.  Note: this is echelon form, 
    not reduced echelon form; this routine does not end the same way as does 
    M.echelon_form().

    """
    num_rows=M.nrows()
    num_cols=M.ncols()
    if augmented_column:
        num_cols-=1

    col = 0   # all cols before this are already done
    for row in range(0,num_rows): 
        # ?Need to swap in a nonzero entry from below
        while (col < num_cols and M[row][col] == 0): 
            for i in M.nonzero_positions_in_column(col):
                if i > row:
                    if printOutput:print(" swap row",row+1,"with row",i+1)
             #       print(M[row])
              #      print(M[i])
              #      print("===================")
                    M.swap_rows(row,i)
                    #print(M)
                    break     
            else:
                col += 1

        if col >= num_cols:
            break

        # Now guaranteed M[row][col] != 0
        if (rescale_leading_entry and M[row][col] != 1):
     #       print(" take",1/M[row][col],"times row",row+1)
            M.rescale_row(row,1/M[row][col])
            #print(M)
        change_flag=False
        for changed_row in range(row+1,num_rows):
            if M[changed_row][col] != 0:
                change_flag=True
                factor=-1*M[changed_row][col]/M[row][col]
                display = not deltas or any(has_delta(M[row][-1], delta) or has_delta(M[changed_row][-1],delta) for delta in deltas)
                if display and printOutput:
                    print(" take",factor,"times row",row+1,"plus row",changed_row+1) 
                    print(M[row])
                    print(M[changed_row])
                    print("->")
                M.add_multiple_of_row(changed_row,row,factor)
                if display and printOutput: print(M[changed_row])
                if len(set(M[changed_row]))==1: print("ZEROED")
        if change_flag:
            pass#print(M)
        col +=1
    print()
    print()
    if printOutput:
        print("Zero Row Count",len([row for row in M if len(set(row))==1]))
        print("Bad rows")
    for row in [row for row in M if row[-1]!=0 and len(set(row))==2]:
        print(row)
        
#Use this to watch the row reduction steps and see what actually happened
#You can specify for it to only output steps involving specific deltas
def test_watch_rref_steps(num_of_players, num_of_strategies, configuration, deltas_to_watch=None):
    G, all_vars = create_game_graph(3,3)
    
    zero_sums, zero_sum_matrix = create_zero_sum_equations(G, True)
    deltas, deltas_matrix = create_delta_equations(G, configuration, True)
    eqs = zero_sum_matrix + deltas_matrix
    eqs_ = convert_eqs_to_solvable_form(G, eqs)
    if deltas_to_watch:
        deltas_ = []
        for eq in eqs:
            if str(eq[-1]) in deltas_to_watch:
                deltas_.append(eq[-1])
        print(deltas_)
    else:
        deltas_ = None
    gauss_method(matrix(eqs_), deltas=deltas_)

from collections import Counter
last_m = None
def find_configs_that_need_algorithm(num_of_players=3, num_of_strategies=3, solutions=None):
 #   print("Create game graph", end="\r\n")
    global found_interesting_case, last_m
    found_interesting_case = False
    G, all_vars = create_game_graph(num_of_players,num_of_strategies)
#    print("Done creating game graph", end="\r\n")
    if not solutions: solutions = known_solutions
    for i, configuration in enumerate(solutions):
        print("Checking",i, configuration, end="\r\n")
        
        zero_sums, zero_sum_matrix = create_zero_sum_equations_specific(G, configuration, True)
        deltas, deltas_matrix = create_delta_equations(G, configuration, True)
     #   print("THe delta rows:", deltas, end="r\n")
        
        eqs = zero_sum_matrix + deltas_matrix        
        #print("Solving",len(eqs),"equations",end="\r\n")
        eqs_ = convert_eqs_to_solvable_form(G, eqs)
        last_m = matrix(eqs_)
        eqs_rref = matrix(eqs_).echelon_form()
        ds__rref = [row[-1] for row in eqs_rref if row[-1]!=0 and len(set(row))==2]
        ds_rref = set(ds__rref)
        zero_rref = [row for row in eqs_rref if len(set(row))==1]
        ends_rref = set(row[-1] for row in eqs_rref)
        inconsistent_rref = [row for row in eqs_rref if len(set(row))==2 and type(row[-1]) in (int, float)]
        if len(inconsistent_rref):
            print(i,configuration, "Inconsistent!!!", end="\r\n")
        if len(ds_rref)==0:
            print(i,configuration,"Zero rows:",len(zero_rref), in_rref(eqs_rref), end="\r\n")
        else:
            print("DS",ds_rref)
            deltas_per_row = [set(qe.variables()) for qe in ds_rref]
            deltas_per_row__ = [set(v) for v in deltas_per_row]
            for i,row in enumerate(deltas_per_row__):
                for j, other in enumerate(deltas_per_row__):
                    if i==j: continue
                    if row.issubset(other):
                        other.difference_update(row)
                            
            unique_var_counts = []
            each_unique = True
            non_unique_indices = []
            for i,row in enumerate(deltas_per_row__):
                my_deltas = set(row)
                for j,other_row in enumerate(deltas_per_row__):
                    if i==j: continue
                    my_deltas.difference_update(other_row)
                    if len(my_deltas)==0:
                        each_unique = False
                        non_unique_indices.append(i)
                        break
                unique_var_counts.append(len(my_deltas))
                
            unique_var_counts = Counter(unique_var_counts)            
            non_unique_overlaps = 0
            for i in non_unique_indices:
                for j in non_unique_indices:
                    if j<=i: continue
                    if any(d in deltas_per_row[j] for d in deltas_per_row[i]):
                        non_unique_overlaps+=1
            
                
                    
                    
            used_deltas = Counter([d for row in deltas_per_row for d in row])
            collisions = []
            for d, count in used_deltas.items():
                if count>1:
                    collisions.append(d)
            
            balanced = True
            for qe in ds_rref:
                pos = 0
                neg = 0
                for v in qe.variables():
                    if qe.coefficient(v)<0:
                        neg+=1
                    else:
                        pos+=1
                if neg!=pos:
                    balanced = False
                    break
                
            
            pairwise_unique_count = []
            pr = True
            for i, row in enumerate(deltas_per_row):
                for j, other in enumerate(deltas_per_row):
                    if i==j: continue
                    unique_i = [d for d in row if d not in other]
                    unique_j = [d for d in other if d not in row]
                    mi = min(len(unique_i), len(unique_j))
            #        if len(unique_i)==0 and len(unique_j)>1:
             #           print("AAAH", row, other)
                    m = max(len(unique_i), len(unique_j))
              #      if m==2 and len(row)>2 and len(other)>2 and pr:
               #         print("Only 2 diff", unique_i, unique_j,"\n", ds__rref[i],"\n", ds__rref[j])
                #        pr=False
                    if mi != 0:
                        pairwise_unique_count.append(mi)
                        pairwise_unique_count.append(m)
            pairwise_unique_count = Counter(pairwise_unique_count)
                    
                    
            print(i, configuration, "needs solving. Solvable rows:", len(ds_rref),"zeroable rows:",len(zero_rref),"each_row_has_unique_delta:",each_unique, "unique_delta_counts:", list(unique_var_counts.items()), "non_unique_overlaps:", non_unique_overlaps, "collisions:",collisions, end="\r\n")
            print("pairwise unique count:",list(pairwise_unique_count.items()),end="\r\n")
            print("Pos/Neg Balanced:",balanced,end="\r\n")
            yield configuration, ds_rref

def is_feasible_quick(num_of_players,num_of_strategies, configuration):
    G, all_vars = create_game_graph(num_of_players, num_of_strategies)
    for config, deltas_to_solve in find_configs_that_need_algorithm(num_of_players, num_of_strategies, [configuration]):
        deqs, ds = convert_multivariate_eqs_to_vars(deltas_to_solve)
        deqs = [eq==0 for eq in deqs]
        for d in ds:
            deqs.append(d>0)
        solution = solve(deqs, *ds)
        if not solution or len(solution)==0:
            return False
        print("Dines algorithm ok:", has_positive_solution([convert_to_delta_map(eq) for eq in deltas_to_solve], 3), end="\r\n")
    return True
            
def test_rref():
    G, all_vars = create_game_graph(3,3)
    zero_sums, zero_sum_matrix = create_zero_sum_equations(G, True)
    for i, configuration in enumerate(known_solutions):
        deltas, deltas_matrix = create_delta_equations(G, configuration, True)
        eqs = zero_sum_matrix + deltas_matrix        
        eqs_ = convert_eqs_to_solvable_form(G, eqs)
        eqs_rref = matrix(eqs_).rref()
        ds_rref = set([row[-1] for row in eqs_rref if row[-1]!=0 and len(set(row))==2])
        zero_rref = [row for row in eqs_rref if len(set(row))==1]
        ends_rref = set(row[-1] for row in eqs_rref)
        eqs__rref =  matrix(row[0:len(row)-1] for row in eqs_).rref()
        zero__rref = [row for row in eqs__rref if len(set(row))==1]
        print(i, configuration, len(zero_rref)==len(zero__rref))
        
def test_echelon():
    G, all_vars = create_game_graph(3,3)
    zero_sums, zero_sum_matrix = create_zero_sum_equations(G, True)
    for i, configuration in enumerate(known_solutions):
        deltas, deltas_matrix = create_delta_equations(G, configuration, True)
        eqs = zero_sum_matrix + deltas_matrix        
        eqs_ = convert_eqs_to_solvable_form(G, eqs)
        eqs_rref = matrix(eqs_).echelon_form()
        print(i, configuration, in_rref(eqs_rref))
        
def test_equation_building(configuration=((0,0,0),)):
    G, all_vars = create_game_graph(3,3)
    deltas, deltas_matrix = create_delta_equations(G, configuration, True)
    for row in deltas_matrix:
        print(row)
    print(len(deltas_matrix), len(set(d[-1] for d in deltas_matrix)))
    
#Helper function for moving matrices to mathematica (to double check answers)
def prepare_for_wolfram(m):
    m_str = str(m)
    m_str = m_str.replace("(", "{")
    m_str = m_str.replace(")", "}")
    m_str = m_str.replace('[', '{')
    m_str = m_str.replace("]", "}")
    m_str = m_str.replace("_", "u")
    return m_str


def dinnesalgorithm(delta_equations):
    
    for i, equation in enumerate(delta_equations):
        deltas = get_deltas(equation)
        
        
print("Test that code still works", end="r\n")
print("Start is feasible", end="\r\n")
#is_feasible_quick(,{(2, 3, 2, 2), (1, 2, 2, 0), (1, 2, 0, 1), (3, 3, 0, 3), (2, 2, 2, 0), (1, 0, 0, 2), (3, 1, 3, 3), (1, 1, 2, 3), (2, 2, 1, 0), (1, 1, 0, 2), (0, 0, 1, 2)})
#print(is_feasible_scratch_rigorous(2,2,[(0,0),(1,1)]))

#print(is_feasible_scratch(3,3,[(0,0,0),(1,0,0),(1,1,0),(0,1,0),(0,0,1),(1,0,1),(1,1,1),(0,1,1)]))#3,4,{(3, 0, 2), (2, 0, 0), (0, 0, 0), (1, 2, 0), (3, 1, 2)}))
print("Quicker",end="\r\n")
#print(is_feasible_quick(3,3,[(0,0,0),(1,0,0),(1,1,0),(0,1,0),(0,0,1),(1,0,1),(1,1,1),(0,1,1)]))#3,4,{(3, 0, 2), (2, 0, 0), (0, 0, 0), (1, 2, 0), (3, 1, 2)}))
print(is_feasible_quick(2,2,[(0,0),(1,1)]))


Start is feasible
Quicker
Checking 0 [(0, 0), (1, 1)]
Canonicals  {(0, 0): (0, 0), (1, 0): (0, 0), (0, 1): (1, 1), (1, 1): (1, 1)}
DS {d_1_v_0_0_0_1 + d_0_v_0_0_1_0 + d_0_v_0_1_1_1 + d_1_v_1_0_1_1}
0 [(0, 0), (1, 1)] needs solving. Solvable rows: 1 zeroable rows: 0 each_row_has_unique_delta: True unique_delta_counts: [(4, 1)] non_unique_overlaps: 0 collisions: []
pairwise unique count: []
Pos/Neg Balanced: False
False


In [ ]:
#This cell will randomly create configurations for a given num_of_players and num_of_strategies and see if it is feasible, and if so, print as much info about it as possibel
import random
def check_search_for_random_subgraph(num_of_players, num_of_strategies, p=0.3, num_of_iterations=10, max_attempts=None):
    G, all_vars = create_game_graph(num_of_players, num_of_strategies)
    attempt = 0
    while True:
        configuration = set()
        
        if max_attempts and attempt == max_attempts-1:
            break
        print("Attempt", attempt, end='\r\n')
        attempt+=1
        i = 0
        while i < num_of_iterations:
            H = G.random_subgraph(p)
            if H.num_verts() <= 1:
                continue
            i+=1
            verts = list(H.vertices())
            configuration.add(random.choice(verts))
            if random.randint(0,100)>33: 
                configuration.add(random.choice(verts))
        print(configuration)
        if is_feasible_quick(num_of_players, num_of_strategies, configuration):
            print("Is feasible", end="\r\n")
            return configuration
        print("Is not feasible",end='\r\n')

for i in range(100):
 #   num_of_players, num_of_strategies = random.choice([(3,3),(3,3),(3,3),(3,3)])
    num_of_players, num_of_strategies = random.choice([(3,6)])
    num_of_iterations = random.choice([2,3,4,5,6,7,8, 9, 10, 11, 12, 13])
    print("Doing",num_of_players,"x",num_of_strategies, "iters:",num_of_iterations)
    check_search_for_random_subgraph(num_of_players, num_of_strategies, num_of_iterations=num_of_iterations)

Doing 3 x 6 iters: 9
Attempt 0
{(5, 0, 1), (1, 5, 3), (3, 1, 1), (5, 1, 2), (1, 4, 0), (4, 4, 0), (5, 1, 1), (1, 4, 5), (2, 1, 5), (4, 1, 0), (4, 0, 3), (4, 3, 3), (1, 4, 2), (3, 1, 3), (3, 0, 1), (4, 2, 3)}
Checking 0 {(5, 0, 1), (1, 5, 3), (3, 1, 1), (5, 1, 2), (1, 4, 0), (4, 4, 0), (5, 1, 1), (1, 4, 5), (2, 1, 5), (4, 1, 0), (4, 0, 3), (4, 3, 3), (1, 4, 2), (3, 1, 3), (3, 0, 1), (4, 2, 3)}
Canonicals  {(0, 0, 1): (5, 0, 1), (1, 5, 1): (5, 0, 1), (2, 5, 0): (5, 0, 1), (0, 5, 3): (1, 5, 3), (1, 1, 3): (1, 5, 3), (2, 1, 5): (1, 5, 3), (0, 1, 1): (3, 1, 1), (1, 3, 1): (3, 1, 1), (2, 3, 1): (3, 1, 1), (0, 1, 2): (5, 1, 2), (1, 5, 2): (5, 1, 2), (2, 5, 1): (5, 1, 2), (0, 4, 0): (1, 4, 0), (1, 1, 0): (1, 4, 0), (2, 1, 4): (1, 4, 0), (1, 4, 0): (4, 4, 0), (2, 4, 4): (4, 4, 0), (0, 4, 5): (1, 4, 5), (1, 1, 5): (1, 4, 5), (0, 1, 5): (2, 1, 5), (1, 2, 5): (2, 1, 5), (2, 2, 1): (2, 1, 5), (0, 1, 0): (4, 1, 0), (2, 4, 1): (4, 1, 0), (0, 0, 3): (4, 0, 3), (1, 4, 3): (4, 0, 3), (2, 4, 0): (4, 0, 3

0 {(4, 0, 1), (2, 3, 5), (0, 3, 3), (3, 5, 4), (0, 2, 1), (0, 2, 0), (1, 4, 4), (1, 0, 5), (1, 3, 1), (2, 5, 1), (5, 2, 1), (1, 4, 2), (5, 2, 2), (0, 5, 4), (5, 1, 3), (2, 1, 3), (1, 3, 3)} Zero rows: 8 True
Is feasible
Doing 3 x 6 iters: 13
Attempt 0
{(0, 5, 5), (2, 2, 5), (4, 3, 5), (4, 3, 2), (5, 2, 1), (4, 3, 3), (3, 3, 0), (5, 2, 2), (2, 4, 3), (0, 2, 2), (5, 1, 0), (3, 1, 3), (2, 5, 0), (5, 0, 2), (2, 1, 5), (5, 3, 2), (3, 2, 5), (0, 4, 1), (4, 2, 1), (1, 1, 5), (1, 3, 4), (1, 0, 2)}
Checking 0 {(0, 5, 5), (2, 2, 5), (4, 3, 5), (4, 3, 2), (5, 2, 1), (4, 3, 3), (3, 3, 0), (5, 2, 2), (2, 4, 3), (0, 2, 2), (5, 1, 0), (3, 1, 3), (2, 5, 0), (5, 0, 2), (2, 1, 5), (5, 3, 2), (3, 2, 5), (0, 4, 1), (4, 2, 1), (1, 1, 5), (1, 3, 4), (1, 0, 2)}
Canonicals  {(0, 5, 5): (0, 5, 5), (1, 0, 5): (0, 5, 5), (2, 0, 5): (0, 5, 5), (0, 2, 5): (2, 2, 5), (1, 2, 5): (2, 2, 5), (2, 2, 2): (2, 2, 5), (0, 3, 5): (4, 3, 5), (1, 4, 5): (4, 3, 5), (2, 4, 3): (4, 3, 5), (0, 3, 2): (4, 3, 2), (1, 4, 2): (4, 3, 

DS {d_0_v_3_2_0_5_2_0 + d_1_v_5_2_0_5_4_0 + d_1_v_3_2_0_3_4_0 + d_0_v_3_4_0_5_4_0 - d_2_v_5_4_3_5_4_5 + d_2_v_5_4_0_5_4_3 + d_2_v_3_2_0_3_2_5 - d_0_v_3_4_5_5_4_5 - d_1_v_5_2_5_5_4_5}
0 {(1, 5, 3), (5, 2, 0), (3, 4, 0), (4, 3, 2), (5, 2, 2), (0, 2, 0), (0, 5, 2), (3, 1, 5), (5, 5, 4), (5, 4, 3), (2, 4, 4), (0, 3, 3), (0, 0, 1), (2, 1, 5), (5, 0, 0), (3, 2, 5), (5, 2, 4), (4, 2, 0), (3, 4, 5), (5, 2, 5)} needs solving. Solvable rows: 1 zeroable rows: 12 each_row_has_unique_delta: True unique_delta_counts: [(9, 1)] non_unique_overlaps: 0 collisions: []
pairwise unique count: []
Pos/Neg Balanced: False
Dines algorithm ok: True
Is feasible
Doing 3 x 6 iters: 8
Attempt 0
{(3, 2, 0), (1, 2, 0), (1, 2, 2), (1, 0, 0), (4, 4, 0), (3, 5, 3), (4, 2, 3), (4, 2, 0), (3, 4, 1), (2, 0, 5), (5, 1, 3)}
Checking 0 {(3, 2, 0), (1, 2, 0), (1, 2, 2), (1, 0, 0), (4, 4, 0), (3, 5, 3), (4, 2, 3), (4, 2, 0), (3, 4, 1), (2, 0, 5), (5, 1, 3)}
Canonicals  {(0, 2, 0): (3, 2, 0), (1, 3, 0): (3, 2, 0), (2, 3, 2): (3,

0 {(5, 5, 3), (1, 1, 0), (2, 4, 0), (4, 2, 2), (0, 3, 3), (3, 5, 0), (5, 5, 2), (1, 3, 1), (2, 1, 5), (0, 0, 5), (1, 3, 0), (5, 2, 2), (4, 1, 5), (2, 5, 2), (4, 0, 5), (5, 2, 5), (1, 0, 2)} Zero rows: 10 True
Is feasible
Doing 3 x 6 iters: 5
Attempt 0
{(4, 5, 2), (2, 4, 0), (0, 4, 1), (0, 5, 3), (5, 4, 2), (3, 4, 4), (5, 2, 1), (2, 1, 2), (5, 3, 2)}
Checking 0 {(4, 5, 2), (2, 4, 0), (0, 4, 1), (0, 5, 3), (5, 4, 2), (3, 4, 4), (5, 2, 1), (2, 1, 2), (5, 3, 2)}
Canonicals  {(0, 5, 2): (4, 5, 2), (1, 4, 2): (4, 5, 2), (2, 4, 5): (4, 5, 2), (0, 4, 0): (2, 4, 0), (1, 2, 0): (2, 4, 0), (2, 2, 4): (2, 4, 0), (0, 4, 1): (0, 4, 1), (1, 0, 1): (0, 4, 1), (2, 0, 4): (0, 4, 1), (0, 5, 3): (0, 5, 3), (1, 0, 3): (0, 5, 3), (2, 0, 5): (0, 5, 3), (0, 4, 2): (5, 4, 2), (1, 5, 2): (5, 4, 2), (2, 5, 4): (5, 4, 2), (0, 4, 4): (3, 4, 4), (1, 3, 4): (3, 4, 4), (2, 3, 4): (3, 4, 4), (0, 2, 1): (5, 2, 1), (1, 5, 1): (5, 2, 1), (2, 5, 2): (5, 2, 1), (0, 1, 2): (2, 1, 2), (1, 2, 2): (2, 1, 2), (2, 2, 1): (2, 1, 

Is not feasible
Attempt 1
{(2, 3, 5), (1, 5, 3), (0, 1, 3), (0, 5, 2), (5, 5, 2), (2, 4, 3), (3, 2, 2), (5, 4, 0), (2, 1, 4), (5, 3, 3), (3, 5, 1), (5, 3, 1), (3, 2, 5), (2, 1, 0), (5, 0, 1), (5, 4, 5), (5, 3, 0), (1, 3, 4), (0, 5, 4)}
Checking 0 {(2, 3, 5), (1, 5, 3), (0, 1, 3), (0, 5, 2), (5, 5, 2), (2, 4, 3), (3, 2, 2), (5, 4, 0), (2, 1, 4), (5, 3, 3), (3, 5, 1), (5, 3, 1), (3, 2, 5), (2, 1, 0), (5, 0, 1), (5, 4, 5), (5, 3, 0), (1, 3, 4), (0, 5, 4)}
Canonicals  {(0, 3, 5): (2, 3, 5), (1, 2, 5): (2, 3, 5), (2, 2, 3): (2, 3, 5), (0, 5, 3): (1, 5, 3), (1, 1, 3): (1, 5, 3), (2, 1, 5): (1, 5, 3), (0, 1, 3): (0, 1, 3), (1, 0, 3): (0, 1, 3), (2, 0, 1): (0, 1, 3), (0, 5, 2): (0, 5, 2), (1, 0, 2): (0, 5, 2), (2, 0, 5): (0, 5, 2), (1, 5, 2): (5, 5, 2), (2, 5, 5): (5, 5, 2), (0, 4, 3): (2, 4, 3), (1, 2, 3): (2, 4, 3), (2, 2, 4): (2, 4, 3), (0, 2, 2): (3, 2, 2), (1, 3, 2): (3, 2, 2), (2, 3, 2): (3, 2, 2), (0, 4, 0): (5, 4, 0), (1, 5, 0): (5, 4, 0), (2, 5, 4): (5, 4, 0), (0, 1, 4): (2, 1, 4), (1

...

0 {(0, 3, 2), (5, 2, 3), (4, 4, 1), (0, 0, 2), (2, 0, 3), (4, 0, 0), (3, 1, 2), (1, 1, 3), (3, 1, 0), (2, 1, 1), (4, 5, 1), (3, 3, 5), (3, 5, 5), (4, 1, 4), (5, 2, 5), (4, 0, 5), (5, 4, 4), (1, 1, 4)} Zero rows: 8 True
Is feasible
Doing 3 x 6 iters: 11
Attempt 0
{(4, 4, 1), (4, 3, 5), (3, 3, 1), (1, 0, 4), (0, 2, 3), (1, 0, 5), (2, 0, 1), (2, 1, 4), (0, 0, 1), (4, 1, 0), (1, 2, 3), (5, 0, 2), (0, 0, 0), (3, 5, 2), (2, 1, 1), (3, 0, 0), (1, 5, 1), (3, 0, 1), (1, 0, 2)}
Checking 0 {(4, 4, 1), (4, 3, 5), (3, 3, 1), (1, 0, 4), (0, 2, 3), (1, 0, 5), (2, 0, 1), (2, 1, 4), (0, 0, 1), (4, 1, 0), (1, 2, 3), (5, 0, 2), (0, 0, 0), (3, 5, 2), (2, 1, 1), (3, 0, 0), (1, 5, 1), (3, 0, 1), (1, 0, 2)}
Canonicals  {(0, 4, 1): (4, 4, 1), (1, 4, 1): (4, 4, 1), (2, 4, 4): (4, 4, 1), (0, 3, 5): (4, 3, 5), (1, 4, 5): (4, 3, 5), (2, 4, 3): (4, 3, 5), (0, 3, 1): (3, 3, 1), (1, 3, 1): (3, 3, 1), (2, 3, 3): (3, 3, 1), (0, 0, 4): (1, 0, 4), (1, 1, 4): (1, 0, 4), (2, 1, 0): (1, 0, 4), (0, 2, 3): (0, 2, 3), (1, 0, 

0 {(0, 2, 0), (2, 2, 5), (1, 4, 4), (0, 3, 0), (2, 1, 4), (1, 0, 5), (0, 4, 0), (1, 0, 1), (1, 3, 5), (0, 3, 1), (5, 0, 3), (2, 3, 4), (5, 1, 3), (4, 4, 2)} Zero rows: 7 True
Is feasible
Doing 3 x 6 iters: 9
Attempt 0
{(3, 1, 2), (2, 2, 1), (3, 4, 0), (2, 4, 2), (4, 3, 0), (0, 5, 1), (4, 1, 4), (2, 0, 1), (2, 4, 5), (3, 4, 5), (2, 3, 4), (3, 4, 2), (1, 2, 5)}
Checking 0 {(3, 1, 2), (2, 2, 1), (3, 4, 0), (2, 4, 2), (4, 3, 0), (0, 5, 1), (4, 1, 4), (2, 0, 1), (2, 4, 5), (3, 4, 5), (2, 3, 4), (3, 4, 2), (1, 2, 5)}
Canonicals  {(0, 1, 2): (3, 1, 2), (1, 3, 2): (3, 1, 2), (2, 3, 1): (3, 1, 2), (0, 2, 1): (2, 2, 1), (1, 2, 1): (2, 2, 1), (2, 2, 2): (2, 2, 1), (0, 4, 0): (3, 4, 0), (1, 3, 0): (3, 4, 0), (2, 3, 4): (3, 4, 0), (0, 4, 2): (2, 4, 2), (1, 2, 2): (2, 4, 2), (2, 2, 4): (2, 4, 2), (0, 3, 0): (4, 3, 0), (1, 4, 0): (4, 3, 0), (2, 4, 3): (4, 3, 0), (0, 5, 1): (0, 5, 1), (1, 0, 1): (0, 5, 1), (2, 0, 5): (0, 5, 1), (0, 1, 4): (4, 1, 4), (1, 4, 4): (4, 1, 4), (2, 4, 1): (4, 1, 4), (0, 0, 1

Is not feasible
Attempt 1
{(2, 3, 5), (2, 2, 5), (1, 0, 1), (3, 0, 5), (2, 4, 3), (0, 5, 3), (2, 3, 3), (5, 5, 5), (3, 2, 2), (0, 3, 3), (4, 5, 5), (2, 1, 2), (5, 0, 3), (3, 5, 2), (1, 5, 5), (4, 2, 4), (4, 1, 2), (2, 5, 4), (0, 3, 5), (1, 3, 5), (0, 1, 0)}
Checking 0 {(2, 3, 5), (2, 2, 5), (1, 0, 1), (3, 0, 5), (2, 4, 3), (0, 5, 3), (2, 3, 3), (5, 5, 5), (3, 2, 2), (0, 3, 3), (4, 5, 5), (2, 1, 2), (5, 0, 3), (3, 5, 2), (1, 5, 5), (4, 2, 4), (4, 1, 2), (2, 5, 4), (0, 3, 5), (1, 3, 5), (0, 1, 0)}
Canonicals  {(0, 3, 5): (2, 3, 5), (1, 2, 5): (2, 3, 5), (2, 2, 3): (2, 3, 5), (0, 2, 5): (2, 2, 5), (2, 2, 2): (2, 2, 5), (0, 0, 1): (1, 0, 1), (1, 1, 1): (1, 0, 1), (2, 1, 0): (1, 0, 1), (0, 0, 5): (3, 0, 5), (1, 3, 5): (3, 0, 5), (2, 3, 0): (3, 0, 5), (0, 4, 3): (2, 4, 3), (1, 2, 3): (2, 4, 3), (2, 2, 4): (2, 4, 3), (0, 5, 3): (0, 5, 3), (1, 0, 3): (0, 5, 3), (2, 0, 5): (0, 5, 3), (0, 3, 3): (2, 3, 3), (0, 5, 5): (5, 5, 5), (1, 5, 5): (5, 5, 5), (2, 5, 5): (5, 5, 5), (0, 2, 2): (3, 2, 2), (1

0 {(3, 0, 2), (3, 3, 3), (5, 4, 0), (4, 0, 1), (4, 4, 1), (1, 3, 2), (2, 3, 2), (2, 1, 4), (1, 4, 4), (0, 0, 1), (0, 4, 0), (0, 0, 5), (2, 5, 1), (1, 2, 3), (4, 1, 5)} Zero rows: 5 True
Is feasible
Doing 3 x 6 iters: 4
Attempt 0
{(2, 0, 2), (1, 0, 1), (5, 4, 2), (1, 3, 4), (0, 1, 0), (0, 0, 3)}
Checking 0 {(2, 0, 2), (1, 0, 1), (5, 4, 2), (1, 3, 4), (0, 1, 0), (0, 0, 3)}
Canonicals  {(0, 0, 2): (2, 0, 2), (1, 2, 2): (2, 0, 2), (2, 2, 0): (2, 0, 2), (0, 0, 1): (1, 0, 1), (1, 1, 1): (1, 0, 1), (2, 1, 0): (1, 0, 1), (0, 4, 2): (5, 4, 2), (1, 5, 2): (5, 4, 2), (2, 5, 4): (5, 4, 2), (0, 3, 4): (1, 3, 4), (1, 1, 4): (1, 3, 4), (2, 1, 3): (1, 3, 4), (0, 1, 0): (0, 1, 0), (1, 0, 0): (0, 1, 0), (2, 0, 1): (0, 1, 0), (0, 0, 3): (0, 0, 3), (1, 0, 3): (0, 0, 3), (2, 0, 0): (0, 0, 3)}
0 {(2, 0, 2), (1, 0, 1), (5, 4, 2), (1, 3, 4), (0, 1, 0), (0, 0, 3)} Zero rows: 0 True
Is feasible
Doing 3 x 6 iters: 12
Attempt 0
{(3, 0, 4), (1, 3, 0), (3, 0, 5), (4, 0, 0), (0, 2, 3), (5, 5, 5), (5, 1, 0), (3, 1, 3

DS {-d_1_v_4_3_3_4_4_3 + d_0_v_0_3_1_4_3_1 + d_1_v_4_3_1_4_4_1 - d_0_v_4_4_3_5_4_3 + d_2_v_0_3_1_0_3_3 - d_0_v_0_4_1_5_4_1 + d_0_v_4_4_1_5_4_1 + d_1_v_0_0_1_0_3_1 - d_1_v_0_0_1_0_4_1 - d_2_v_0_4_1_0_4_3 + d_2_v_4_4_1_4_4_4 - d_2_v_4_4_3_4_4_4, -d_2_v_4_3_2_4_3_3 - d_1_v_4_3_3_4_4_3 + d_0_v_0_3_2_3_3_2 - d_0_v_3_3_2_4_3_2 - d_0_v_4_4_3_5_4_3 + d_2_v_0_3_2_0_3_3 - d_1_v_4_1_2_4_3_2 + d_1_v_4_1_2_4_4_2 + d_1_v_0_3_2_0_4_2 + d_0_v_0_4_2_4_4_2 + d_2_v_4_4_2_4_4_4 - d_2_v_4_4_3_4_4_4}
1 {(4, 3, 3), (3, 3, 0), (0, 1, 5), (4, 3, 1), (3, 3, 2), (0, 2, 3), (1, 0, 5), (5, 4, 3), (0, 3, 3), (5, 4, 1), (2, 1, 4), (0, 0, 1), (0, 4, 3), (5, 4, 4), (4, 1, 2), (0, 4, 2), (4, 4, 4), (2, 2, 0), (1, 1, 4)} needs solving. Solvable rows: 2 zeroable rows: 11 each_row_has_unique_delta: True unique_delta_counts: [(9, 2)] non_unique_overlaps: 0 collisions: [d_2_v_4_4_3_4_4_4, d_1_v_4_3_3_4_4_3, d_0_v_4_4_3_5_4_3]
pairwise unique count: [(9, 4)]
Pos/Neg Balanced: True
1
d_0_v_0_3_1_4_3_1+d_1_v_4_3_1_4_4_1+d_2_v_

In [11]:
last_m.str()

'[                0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 1                 1                 1                 0                 0                 0                 0         

In [5]:
is_feasible_quick(3,3,{(2, 2, 2), (1, 2, 1), (0, 1, 1), (0, 0, 2), (1, 2, 2), (2, 1, 1), (0, 0, 1), (2, 0, 2), (1, 0, 1), (1, 0, 2)})

Checking 0 {(1, 2, 1), (2, 2, 2), (0, 1, 1), (0, 0, 2), (1, 2, 2), (2, 1, 1), (0, 0, 1), (2, 0, 2), (1, 0, 1), (1, 0, 2)}
Canonicals  {(0, 2, 1): (1, 2, 1), (1, 1, 1): (1, 2, 1), (2, 1, 2): (1, 2, 1), (0, 2, 2): (2, 2, 2), (1, 2, 2): (2, 2, 2), (2, 2, 2): (2, 2, 2), (0, 1, 1): (0, 1, 1), (1, 0, 1): (0, 1, 1), (2, 0, 1): (0, 1, 1), (0, 0, 2): (0, 0, 2), (1, 0, 2): (0, 0, 2), (2, 0, 0): (0, 0, 2), (1, 1, 2): (1, 2, 2), (1, 2, 1): (2, 1, 1), (2, 2, 1): (2, 1, 1), (0, 0, 1): (0, 0, 1), (2, 2, 0): (2, 0, 2), (2, 1, 0): (1, 0, 1)}
THe delta rows: [v_16_2 - v_17_2 == 0, v_16_0 - v_7_0 == d_0_v_0_2_1_1_2_1, v_16_0 - v_25_0 == d_0_v_1_2_1_2_2_1, v_10_1 - v_16_1 == 0, -v_13_1 + v_16_1 == d_1_v_1_1_1_1_2_1, -v_15_2 + v_16_2 == d_2_v_1_2_0_1_2_1, v_17_0 - v_26_0 == 0, v_20_1 - v_26_1 == 0, -v_23_1 + v_26_1 == d_1_v_2_1_2_2_2_2, v_26_0 - v_8_0 == d_0_v_0_2_2_2_2_2, -v_24_2 + v_26_2 == d_2_v_2_2_0_2_2_2, -v_25_2 + v_26_2 == d_2_v_2_2_1_2_2_2, v_1_1 - v_4_1 == 0, -v_3_2 + v_4_2 == d_2_v_0_1_0_0_1_1, 

True

In [5]:
is_feasible_scratch_rigorous(3,3,{(2, 2, 2), (1, 2, 1), (0, 1, 1), (0, 0, 2), (1, 2, 2), (2, 1, 1), (0, 0, 1), (2, 0, 2), (1, 0, 1), (1, 0, 2)})

True

In [3]:
is_feasible_quick(3,3,[(1, 1, 0), (0, 2, 1), (1, 2, 0), (0, 0, 2), (0, 1, 0), (2, 2, 1), (2, 0, 2), (0, 0, 1), (1, 0, 1), (1, 1, 2), (0, 0, 0), (1, 0, 2)])

Checking 0 [(1, 1, 0), (0, 2, 1), (1, 2, 0), (0, 0, 2), (0, 1, 0), (2, 2, 1), (2, 0, 2), (0, 0, 1), (1, 0, 1), (1, 1, 2), (0, 0, 0), (1, 0, 2)]
DS {-d_2_v_1_0_0_1_0_1 + d_2_v_1_0_0_1_0_2, -d_0_v_0_0_1_2_0_1 + d_0_v_1_0_1_2_0_1, -d_0_v_0_2_1_1_2_1 + d_0_v_1_2_1_2_2_1, -d_1_v_1_0_0_1_1_0 + d_1_v_1_0_0_1_2_0, -d_1_v_0_1_0_0_2_0 + d_1_v_0_0_0_0_2_0, d_1_v_1_0_0_1_1_0 + d_0_v_0_2_1_1_2_1 + d_2_v_0_2_0_0_2_1 + d_2_v_1_2_0_1_2_1 + d_0_v_0_2_0_1_2_0 + d_1_v_0_1_0_0_2_0 + d_1_v_1_0_1_1_2_1 + d_2_v_1_0_0_1_0_1 + d_0_v_0_0_0_1_0_0, -d_0_v_0_2_1_1_2_1 - d_2_v_0_2_0_0_2_1 - d_2_v_1_2_0_1_2_1 - d_0_v_0_2_0_1_2_0 - d_1_v_0_0_2_0_1_2 - d_2_v_0_1_0_0_1_2 - d_1_v_0_1_0_0_2_0 - d_1_v_1_0_1_1_2_1 - d_0_v_0_1_2_1_1_2, d_0_v_1_1_0_2_1_0 - d_0_v_0_1_0_2_1_0, -d_2_v_1_1_0_1_1_1 + d_2_v_1_1_1_1_1_2, -d_1_v_0_1_1_0_2_1 + d_1_v_0_0_1_0_1_1, -d_0_v_0_2_1_1_2_1 - d_2_v_0_2_0_0_2_1 - d_2_v_1_2_0_1_2_1 - d_0_v_0_2_0_1_2_0 - d_1_v_0_0_2_0_1_2 - d_2_v_0_1_0_0_1_2 - d_1_v_0_1_0_0_2_0 - d_1_v_1_0_1_1_2_1 - d_1_v_1_1_2_1

False

In [24]:
G,all_vars = create_game_graph(3,3)
configuration={(0,0,0)}        
zero_sums, zero_sum_matrix = create_zero_sum_equations_specific(G, configuration, True)
deltas, deltas_matrix = create_delta_equations(G, configuration, True)
eqs = zero_sum_matrix + deltas_matrix        
print(str(matrix(eqs)))

[                1                 1                 1                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0                 0          

In [ ]:
is_feasible_quick(3,5,{(3, 0, 2), (4, 3, 4), (0, 4, 4), (4, 4, 0), (3, 0, 4), (1, 0, 1), (4, 0, 3), (3, 4, 1), (1, 4, 2), (0, 1, 4), (3, 3, 0), (4, 3, 0), (2, 3, 1), (1, 4, 3), (2, 4, 0), (1, 4, 0), (3, 3, 2), (2, 4, 3), (2, 4, 2), (2, 0, 1), (3, 1, 3), (0, 3, 2), (3, 2, 0), (3, 1, 1), (2, 0, 3), (0, 3, 0), (2, 1, 4), (0, 0, 1), (4, 1, 0), (1, 2, 4), (2, 1, 2), (4, 2, 4), (3, 2, 4), (2, 1, 3), (1, 1, 0), (2, 2, 2), (0, 3, 4), (4, 2, 1), (2, 2, 0), (3, 0, 0), (3, 4, 4), (4, 2, 0), (1, 3, 4), (3, 0, 1)})

### 